In [5]:
import operator
from typing import Annotated
from pydantic import BaseModel, Field

In [4]:
import os
from dotenv import load_dotenv


load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "DSLGENAGENT"

In [7]:
ROLES = {
    "1": {
        "name": "一般知識エキスパート",
        "description": "幅広い分野の一般的な質問に答える",
        "details": "幅広い分野の一般的な質問に対して、正確で分かりやすい回答を提供してください。"
    },
    "2": {
        "name": "生成AI製品エキスパート",
        "description": "生成AIや関連製品、技術に関する専門的な質問に答える",
        "details": "生成AIや関連製品、技術に関する専門的な質問に対して、最新の情報と深い洞察を提供してください。"
    },
    "3": {
        "name": "カウンセラー",
        "description": "個人的な悩みや心理的な問題に対してサポートを提供する",
        "details": "個人的な悩みや心理的な問題に対して、共感的で支援的な回答を提供し、可能であれば適切なアドバイスも行ってください。"
    }
}

In [8]:
import operator
from typing import Annotated

from pydantic import BaseModel, Field


class State(BaseModel):
    query: str = Field(..., description="ユーザーからの質問")
    current_role: str = Field(
        default="", description="選定された回答ロール"
    )
    messages: Annotated[list[str], operator.add] = Field(
        default=[], description="回答履歴"
    )
    current_judge: bool = Field(
        default=False, description="品質チェックの結果"
    )
    judgement_reason: str = Field(
        default="", description="品質チェックの判定理由"
    )

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import ConfigurableField

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
# 後からmax_tokensの値を変更できるように、変更可能なフィールドを宣言
llm = llm.configurable_fields(max_tokens=ConfigurableField(id='max_tokens'))

In [25]:
from typing import Any

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def selection_node(state: State) -> dict[str, Any]:
    query = state.query
    role_options = "\n".join([f"{k}. {v['name']}: {v['description']}" for k, v in ROLES.items()])
    prompt = ChatPromptTemplate.from_template(
"""質問を分析し、最も適切な回答担当ロールを選択してください。

選択肢:
{role_options}

回答は選択肢の番号（1、2、または3）のみを返してください。

質問: {query}
""".strip()
    )
    # 選択肢の番号のみを返すことを期待したいため、max_tokensの値を1に変更
    chain = prompt | llm.with_config(configurable=dict(max_tokens=1)) | StrOutputParser()
    role_number = chain.invoke({"role_options": role_options, "query": query})

    selected_role = ROLES[role_number.strip()]["name"]
    return {"current_role": selected_role}

In [20]:
def answering_node(state: State) -> dict[str, Any]:
    query = state.query
    role = state.current_role
    role_details = "\n".join([f"- {v['name']}: {v['details']}" for v in ROLES.values()])
    prompt = ChatPromptTemplate.from_template(
"""あなたは{role}として回答してください。以下の質問に対して、あなたの役割に基づいた適切な回答を提供してください。

役割の詳細:
{role_details}

質問: {query}

回答:""".strip()
    )
    chain = prompt | llm | StrOutputParser()
    answer = chain.invoke({"role": role, "role_details": role_details, "query": query})
    return {"messages": [answer]}

In [21]:
class Judgement(BaseModel):
    judge: bool = Field(default=False, description="判定結果")
    reason: str = Field(default="", description="判定理由")

def check_node(state: State) -> dict[str, Any]:
    query = state.query
    answer = state.messages[-1]
    prompt = ChatPromptTemplate.from_template(
"""以下の回答の品質をチェックし、問題がある場合は'False'、問題がない場合は'True'を回答してください。
また、その判断理由も説明してください。

ユーザーからの質問: {query}
回答: {answer}
""".strip()
    )
    chain = prompt | llm.with_structured_output(Judgement)
    result: Judgement = chain.invoke({"query": query, "answer": answer})

    return {
        "current_judge": result.judge,
        "judgement_reason": result.reason
    }

In [22]:
from langgraph.graph import StateGraph

workflow = StateGraph(State)
workflow.add_node("selection", selection_node)
workflow.add_node("answering", answering_node)
workflow.add_node("check", check_node)
# selectionノードから処理を開始
workflow.set_entry_point("selection")
# selectionノードからansweringノードへ
workflow.add_edge("selection", "answering")
# answeringノードからcheckノードへ
workflow.add_edge("answering", "check")
from langgraph.graph import END

# checkノードから次のノードへの遷移に条件付きエッジを定義
# state.current_judgeの値がTrueならENDノードへ、Falseならselectionノードへ
workflow.add_conditional_edges(
    "check",
    lambda state: state.current_judge,
    {True: END, False: "selection"}
)

In [23]:
compiled = workflow.compile()


In [24]:
initial_state = State(query="生成AIについて教えてください")
result = compiled.invoke(initial_state)

input_variables=['query', 'role_options'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query', 'role_options'], input_types={}, partial_variables={}, template='質問を分析し、最も適切な回答担当ロールを選択してください。\n\n選択肢:\n{role_options}\n\n回答は選択肢の番号（1、2、または3）のみを返してください。\n\n質問: {query}'), additional_kwargs={})]


In [16]:
result

{'query': '生成AIについて教えてください',
 'current_role': '生成AI製品エキスパート',
 'messages': ['生成AIとは、人工知能の一分野であり、特にテキスト、画像、音声などのコンテンツを自動的に生成する技術を指します。これらの技術は、機械学習や深層学習のアルゴリズムを活用しており、特に大規模なデータセットから学習することで、創造的な出力を生み出すことができます。\n\n### 主な特徴と技術\n1. **自然言語処理（NLP）**: テキスト生成においては、GPT（Generative Pre-trained Transformer）などのモデルが広く使用されています。これらのモデルは、文脈を理解し、流暢で意味のある文章を生成する能力があります。\n\n2. **画像生成**: DALL-EやMidjourneyなどのモデルは、テキストから画像を生成することができます。これにより、ユーザーは具体的な指示を与えることで、独自のビジュアルコンテンツを作成できます。\n\n3. **音声生成**: 音声合成技術も進化しており、特定の声やスタイルで音声を生成することが可能です。これにより、ナレーションや対話型AIの開発が進んでいます。\n\n### 利用例\n- **コンテンツ制作**: ブログ記事、広告コピー、ソーシャルメディアの投稿などの自動生成。\n- **デザイン**: グラフィックデザインやプロダクトデザインのアイデア出し。\n- **教育**: 学習教材の生成や個別指導のためのカスタマイズされた問題作成。\n\n### 課題と倫理\n生成AIには、著作権や偽情報の拡散、バイアスの問題など、いくつかの倫理的な課題も伴います。これらの問題に対処するためには、透明性のある開発と利用が求められています。\n\n生成AIは、今後もさまざまな分野での応用が期待されており、技術の進化とともに新たな可能性が広がっています。'],
 'current_judge': True,
 'judgement_reason': '回答は生成AIについての基本的な定義、主な特徴、利用例、課題と倫理に関する情報を包括的に提供しており、内容が正確で関連性が高い。'}

# Agent tool のテスト

In [4]:
from langchain_core.tools import tool

# Toolデコレーターを関数に付ける
@tool
def add(a: int, b: int) -> int:
    """2つの値を足し算して返す"""
    return a + b

print(add.invoke({'a': 3, 'b': 4}))


7


In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        (
            'system',
            """
            与えられたメッセージに従って計算処理を呼び出してください

            """,
        ),
        ('placeholder', '{messages}'),
    ]
)
# bind_toolsで、toolを紐づける
chain = prompt | ChatOpenAI().bind_tools([add])
print(chain.invoke({'messages': ['3 + 4']}))

content='' additional_kwargs={'tool_calls': [{'id': 'call_ZHKU98c2XiRbmtXuiOOXIvWT', 'function': {'arguments': '{"a":3,"b":4}', 'name': 'add'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 92, 'total_tokens': 110, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None} id='run-45c923ad-9161-4b7d-bfb4-f49bb517c577-0' tool_calls=[{'name': 'add', 'args': {'a': 3, 'b': 4}, 'id': 'call_ZHKU98c2XiRbmtXuiOOXIvWT', 'type': 'tool_call'}] usage_metadata={'input_tokens': 92, 'output_tokens': 18, 'total_tokens': 110, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from typing_extensions import Annotated

# toolの定義
@tool
def add(
    a: Annotated[int, '一つ目の値'],
    b: Annotated[int, '二つ目の値'],
) -> int:
    """2つの値を足し算して返す"""
    return a + b + a + b

# プロンプトの定義
prompt = ChatPromptTemplate.from_messages(
    [
        (
            'system',
            '与えられたinputに従って計算処理を呼び出してください',
        ),
        ('placeholder', '{messages}'),
    ]
)

# エージェントの作成
agent = create_react_agent(
    model=ChatOpenAI(model='gpt-4o-mini'),
    tools=[add],
    state_modifier=prompt
)

# エージェントの実行
result = agent.invoke({'messages': ['3 + 4の計算結果は？']})
print(result['messages'][-1].content)
# => 3 + 4の計算結果は7です。

3 + 4の計算結果は14です。


In [26]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from typing_extensions import Annotated
from langchain.agents import initialize_agent, Tool
from langchain.agents.mrkl import prompt

# Toolデコレーターを関数に付ける
@tool
def start_node(
    input: Annotated[str, 'input形式']
    ) -> str:
    """start node の仕様を返す"""

    start_node_context = """
      ## start_node specification:
          start_node:
            - id: ユニークなID
            - type: start
            - variables:
              # ファイル入力の例
              - type: file
                variable: input_document
                label: ドキュメント
                required: true
                max_length: 48
                allowed_file_types:
                  - document
                allowed_file_upload_methods:
                  - local_file
                  - remote_url

              # 数値入力の例
              - type: number
                variable: input_number
                label: 数値
                required: true
                min: 0
                max: 1000000

              # 段落入力の例
              - type: paragraph
                variable: danraku
                label: 段落
                required: true
                max_length: 48
                options: []

              # 短文入力の例
              - type: text-input
                variable: tanbun
                label: 短文
                required: true
                max_length: 48
                options: []    

      ## start_node description:
      start node
         ユースケース:
         - ワークフローの開始点として機能
         - ユーザーからの入力を受け付け
         - 入力された情報を変数として保存

         構造:
         - id: ユニークなID（必須）
         - type: start（固定）
         - variables: 入力変数の配列（必須）
           - type: 変数の型（必須）
           - variable: 変数名（必須）
           - label: 表示ラベル（必須）
           - required: 必須入力かどうか（必須）
           - その他設定（型に応じて必要）

         変数型と固有設定:
         - file（ファイル入力）:
           - allowed_file_types: 許可するファイルタイプ
             - document: ドキュメントファイル
             - image: 画像ファイル
             - audio: 音声ファイル
             - video: 動画ファイル
           - allowed_file_upload_methods: アップロード方法
             - local_file: ローカルファイル
             - remote_url: リモートURL
           - max_length: 最大ファイル名長
           - required: 必須項目かどうか
         - number（数値入力）:
           - min: 最小値（オプション）
           - max: 最大値（オプション）
         - paragraph（段落テキスト）:
           - max_length: 最大文字数（オプション）
           - options: 選択肢（オプション）
         - text-input（短文テキスト）:
           - max_length: 最大文字数（オプション）
           - options: 選択肢（オプション）              
      """
    return start_node_context


In [27]:

@tool
def end_node(
    input: Annotated[str, 'input形式']
    ) -> str:
    """ end node の仕様を返す"""
    context = """
      ## end_node specification:
          end_node:
            - id: ユニークなID
            - type: end
            - outputs: #出力変数の定義
              - input_data: 入力データ
              - generated_text: 生成されたテキスト
      ## end_node description:
      end node
         ユースケース:
         - ワークフローの終了点として機能
         - 処理結果の出力を定義
         - 後続システムへのデータ受け渡し

         構造:
         - id: ユニークなID（必須）
         - type: end（固定）
         - outputs:（必須）
           - value_selector:（必須）
             - [ノードID]
             - [変数名]
           - variable: 出力変数名（必須）

         特記事項:
         - 少なくとも1つの出力変数が必要
         - 複数の出力変数を定義可能
    """
    return context

@tool
def template_node(
    input: Annotated[str, 'input形式']
    ) -> int:
    """ template node の仕様を返す"""
    context = """
      ## template_transform_node specification:
          template_transform_node:
            - id: ユニークなID
            - type: template-transform
            - data:
                title: テンプレート変換
                template: テンプレート文字列（Jinja2形式）
                variables:
                  - value_selector:  # 入力変数の指定
                    - [入力ノードID]
                    - [変数名]
                    variable: [テンプレート内で使用する変数名]
                outputs:
                  output:  # 固定の出力変数名
                    type: string  # 常にstring型
      ## template_transform_node description:
      Template Transformノード
          ユースケース:
          - テンプレート文字列への変数埋め込み
          - 動的なメッセージ生成
          - 条件付きテキスト生成
          - 配列データの文字列化

          構造:
          - id: ユニークなID（必須）
          - type: template-transform（固定）
          - data:
              title: ノードのタイトル（必須）
              template: Jinja2形式のテンプレート（必須）
              variables: 入力変数の配列（必須）
                - value_selector: 変数の参照元
                  - [入力ノードID]
                  - [変数名]
                  variable: テンプレート内で使用する変数名

          テンプレート構文:
          - 変数参照: 
            - 基本形: {{ 変数名 }}
            - フィルター使用: {{ 変数名|upper }}
          - 条件分岐:
            ```
            {% if 条件 %}
              条件成立時の文字列
            {% else %}
              条件不成立時の文字列
            {% endif %}
            ```
          - ループ処理:
            ```
            {% for item in items %}
              {{ item }}
            {% endfor %}
            ```

          入力変数の型:
          - string: 文字列
          - number: 数値
          - object: オブジェクト
          - array: 配列
          - arrayNumber: 数値配列
          - arrayString: 文字列配列
          - arrayObject: オブジェクト配列

          出力:
          - output: 生成された文字列（string型固定）
        """
    return context

@tool
def llm_node(
    input: Annotated[str, 'input形式']
    ) -> int:
    """ llm node の仕様を返す"""
    context = """
        nodes:
          llm_node:
            - id: ユニークなID
            - type: llm
            - model:
              - provider: openai
              - name: gpt-4o
              - mode: chat
              - completion_params:
                - temperature: 生成時の温度設定
            - prompt_template:
              - id: ユニークなID(例'prompt1')
                role: system
                text: シングートで括ったプロンプトテキスト
            - context:
              - enabled: true
              - variable_selector: 使用する変数の指定
            - vision:
              - enabled: false

      ## llm_node description:
      LLMノード
         ユースケース:
         - テキスト生成や応答の作成
         - 入力テキストの加工や変換
         - 質問への回答生成
         構造:
         - id: ユニークなID（必須）
         - type: llm（固定）
         - model:（必須）
           - provider: openai（固定）
           - name: gpt-4o（固定）
           - mode: chat（固定）
           - completion_params:
             - temperature: 0.0-1.0の値（必須）
         - prompt_template:（必須）
           - role: system（固定）
           - text: プロンプトテキスト（必須）
         - context:（必須）
           - enabled: true（固定）
           - variable_selector: 使用する変数の指定（必須）
         - vision:
           - enabled: false（固定）
         特記事項:
         - プロンプトはシングルクォートで括る必要あり
         - システムロールのプロンプトのみ使用可能
         - 変数参照形式: {{#ノードID.変数名#}}
         - contextのvariable_selectorで指定した変数は、prompt_templateのtextでも使用して下さい。
         
              
    """
    return context

In [28]:
message = """
app:
    mode: workflow
    name: Simple Template Workflow
    version: 0.1.5

workflow:
    graph:
        edges:
        - source: 1700000000001  # StartノードID
          target: 1700000000002  # Template-TransformノードID
          data:
            sourceType: start
            targetType: template-transform
        - source: 1700000000002  # Template-TransformノードID
          target: 1700000000003  # EndノードID
          data:
            sourceType: template-transform
            targetType: end"""


# プロンプトの定義
prompt = ChatPromptTemplate.from_messages(
    [
        (
            'system',
            '与えられたワークフローに従ってノードの情報を呼び出してください',
        ),
        ('placeholder', '{messages}'),
    ]
)

# エージェントの作成
agent = create_react_agent(
    model=ChatOpenAI(model='gpt-4o-mini'),
    tools=[start_node, end_node, template_node, llm_node],
    state_modifier=prompt
)

# エージェントの実行
result = agent.invoke({'messages': [message]})


In [29]:
print(result['messages'][-1].content)


以下は、ワークフロー内の各ノードの情報です：

### Start Node Specification
- **ID**: ユニークなID
- **Type**: start
- **Variables**:
  - **File Input**
    - Type: file
    - Variable: input_document
    - Label: ドキュメント
    - Required: true
    - Max Length: 48
    - Allowed File Types: document
    - Allowed File Upload Methods: local_file, remote_url
  - **Number Input**
    - Type: number
    - Variable: input_number
    - Label: 数値
    - Required: true
    - Min: 0
    - Max: 1000000
  - **Paragraph Input**
    - Type: paragraph
    - Variable: danraku
    - Label: 段落
    - Required: true
    - Max Length: 48
    - Options: []
  - **Text Input**
    - Type: text-input
    - Variable: tanbun
    - Label: 短文
    - Required: true
    - Max Length: 48
    - Options: []

### Template Transform Node Specification
- **ID**: ユニークなID
- **Type**: template-transform
- **Data**:
  - **Title**: テンプレート変更
  - **Template**: テンプレート文字列（Jinja2形式）
  - **Variables**:
    - Value Selector:
      - [入力ノードID]
      - [変数名]
      - 

In [ ]:
workflow_prompt ="""
    output_format: |
      生成されるYAMLファイルは以下の形式に従ってください：
      ```yaml
      app:
        mode: workflow
        name: [ワークフロー名]
        version: 0.1.5

      workflow:
        graph:
          edges:
            # IF/ELSE分岐のエッジ例
            - source: [IF/ELSEノードID]
              target: [ターゲットノードID]
              data:
                sourceType: if-else
                targetType: [ターゲットノードタイプ]
              sourceHandle: 'true'  # IF条件成立時
            - source: [IF/ELSEノードID]
              target: [別のターゲットノードID]
              data:
                sourceType: if-else
                targetType: [ターゲットノードタイプ]
              sourceHandle: 'false'  # ELSE条件時

          nodes:
            - id: [開始ノードID]
              data:
                type: start
                title: 開始
                variables:
                  # ファイル入力の例
                  - type: file
                    variable: input_document
                    label: ドキュメント
                    required: true
                    max_length: 48
                    allowed_file_types:
                      - document
                    allowed_file_upload_methods:
                      - local_file
                      - remote_url

                  # その他の入力タイプ
                  - type: text-input
                    variable: [変数名]
                    type: string/number
                    label: [入力フィールドのラベル]
                    required: true
                    max_length: [最大文字数]

            - id: [LLMノードID]
              data:
                type: llm
                title: LLM
                model:
                  provider: openai
                  name: gpt-4o
                  mode: chat
                  completion_params:
                    temperature: 0.7
                prompt_template:
                  - id: [プロンプトID]
                    role: system
                    text: '[プロンプトテキスト]'
                context:
                  enabled: true
                  variable_selector:
                    - [開始ノードID]
                    - [変数名]
                vision:
                  enabled: false
            - id: [終了ノードID]
              data:
                type: end
                title: 終了
                outputs:
                  - value_selector:
                      - [開始ノードID]
                      - [変数名]
                    variable: inputData
                  - value_selector:
                      - [LLMノードID]
                      - text
                    variable: generatedText
            - id: [テンプレートノードID]
              data:
                type: template-transform
                title: テンプレート変換
                template: |
                  こんにちは、{{ user_name }}さん！
                  あなたの得点は{{ score }}点です。
                  {% if score >= 80 %}
                  合格です！おめでとうございます。
                  {% else %}
                  残念ながら不合格です。
                  {% endif %}
                variables:
                  - value_selector:
                      - [入力ノードID]
                      - user_name
                    variable: user_name
                  - value_selector:
                      - [入力ノードID]
                      - score
                    variable: score
"""

In [ ]:
# プロンプトの定義
prompt = ChatPromptTemplate.from_messages(
    [
        (
            'system',
            '与えられたワークフローに従ってノードの情報を呼び出してください',
        ),
        ('placeholder', '{messages}'),
    ]
)

# エージェントの作成
agent = create_react_agent(
    model=ChatOpenAI(model='gpt-4o-mini'),
    tools=[start_node, end_node, template_node, llm_node],
    state_modifier=prompt
)

# エージェントの実行
result = agent.invoke({'messages': [message]})

In [17]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# toolの定義
@tool
def add(
    a: Annotated[int, '一つ目の値'],
    b: Annotated[int, '二つ目の値'],
) -> int:
    """2つの値を足し算して返す"""
    return a + b

# プロンプトの定義
prompt = ChatPromptTemplate.from_messages(
    [
        (
            'system',
            '与えられたinputに従って計算処理を呼び出してください',
        ),
        ('human', '{input}'),
        # Placeholders fill up a **list** of messages
        ('placeholder', '{agent_scratchpad}'),
    ]
)

# エージェントを作成
agent = create_tool_calling_agent(ChatOpenAI(model='gpt-3.5-turbo'), [add], prompt)

# エージェントを実行
agent_executor = AgentExecutor(agent=agent, tools=[add])
result = agent_executor.invoke({'input': '3 + 4の計算結果は？'})
print(result)
# => {'input': '3 + 4の計算結果は？', 'output': '3 + 4の計算結果は、7です。'}

{'input': '3 + 4の計算結果は？', 'output': '3 + 4の計算結果は7です。'}


In [1]:
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, Tool
from langchain.agents.mrkl import prompt

# Toolデコレーターを関数に付ける
@tool
def weather_function(location):
    match location:
      case "東京" | "Tokyo":
        weather = "晴れ"
      case "大阪" | "Osaka":
        weather = "曇り"
      case "北海道" | "Hokkaido":
        weather = "雪"
      case _ :
        weather = "不明"

    weather_answer = [
        {"天気": weather}
    ]

    return json.dumps(weather_answer)


def lang_chain_agent(text):
    llm = OpenAI(model_name='gpt-4o-mini')
    # toolsに利用したいToolを格納する。LangChainで用意されたToolを利用することも出来る。
    # 代表的なTool一覧：https://book.st-hakky.com/docs/agents-of-langchain/
    tools = [
        Tool(
            name = "Weather",
            func=weather_function,
            description="天気を知りたい場所を入力。例: 東京",
        )
    ]

    # エージェントの準備
    agent = initialize_agent(
        tools,
        llm,
        agent="zero-shot-react-description",
        # AIの回答を日本語にするために必要
        agent_kwargs=dict(suffix='Answer should be in Japanese.' + prompt.SUFFIX), 
        # 以下2行を有効にしておくことで、agentの動きを確認しやすい
        # AIの回答のみ欲しい場合はFalseにする
        verbose=True,
        return_intermediate_steps=True)

    response = agent({"input": text})

    return response


ModuleNotFoundError: No module named 'langchain'